In [1]:
# John Elton jre2132
# Patrick Rogan psr2125
# Joshua Safyan jds2258
# Team 'JJ Reps'
# COMS 4721 Machine Learning
# Course Project

import scipy as sp
import numpy as np
import pandas as pd
import random
import math
from sklearn.qda import QDA
from sklearn.lda import LDA
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import svm

sp.random.seed(1)

##############################################################
# Data Ingestion / Processing
##############################################################
 
data_in = pd.read_csv("data.csv")
test_in = pd.read_csv("quiz.csv")
 
 
categorical_vars = [0, 5, 7, 8, 9, 14, 16, 17, 18, 20, 23, 25, 26, 56, 57, 58]
non_binary = [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 59, 60]
categorical_vars = list(map(str, categorical_vars))
non_binary = list(map(str, non_binary))

def dummies_and_standardize(data, test):
	# see: http://stackoverflow.com/questions/34170413/possible-ways-to-do-one-hot-encoding-in-scikit-learn
    X = pd.get_dummies(data[data.columns[:-1]])
    test = pd.get_dummies(test)

	# get the columns in train that are not in test
    cols_to_add = np.setdiff1d(X.columns, test.columns)

	# add these columns to test, setting them equal to zero
    for col in cols_to_add:
        test[col] = 0

	# select and reorder the test columns using the train columns
    test = test[X.columns]

	#labels
    y = data[data.columns[-1]]
    return X, y, test


data, labels, quiz = dummies_and_standardize(data_in, test_in)
print 1

1


In [2]:
#scale non-binary numerical features to be between 0 and 1
data[non_binary] = data[non_binary].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x))
quiz[non_binary] = quiz[non_binary].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x))

# Get N and dim values for data and quiz
quiz = quiz.values
N = data.shape[0]
dim = data.shape[1] 
N_quiz = quiz.shape[0]
dim_quiz = quiz.shape[1]
print 1

1


In [3]:
# convert the data to sparse array format to reduce memory usage and increase speed
data_sparse = sp.sparse.csr_matrix(data, shape=None, dtype=None, copy=False)
quiz_sparse = sp.sparse.csr_matrix(quiz, shape=None, dtype=None, copy=False)
del data
del quiz
print 1

1


In [4]:
Logistic = LogisticRegression(C=1.0)
#scores_logistic = cross_validation.cross_val_score(Logistic, data_sparse, labels, cv=5)
Logistic.fit(data_sparse, labels)
preds_logistic = Logistic.predict(data_sparse)
print 1

1


In [ ]:
preds_logistic_quiz = Logistic.predict(quiz_sparse)

In [ ]:
RandomForest = RandomForestClassifier(n_estimators=40, criterion='gini', max_features='sqrt', n_jobs=-1,
                                       min_samples_split = 4)
RandomForest.fit(data_sparse, labels)

print 1

In [ ]:
preds = RandomForest.predict(data_sparse)

In [ ]:
preds_quiz = RandomForest.predict(quiz_sparse)

In [ ]:
rf_probs = RandomForest.predict_proba(data_sparse)


In [ ]:
logistic_probs = Logistic.predict_proba(data_sparse)


In [ ]:
rf_probs_quiz = RandomForest.predict_proba(quiz_sparse)
logistic_probs_quiz = Logistic.predict_proba(quiz_sparse)

In [ ]:
preds_new = sp.zeros(N)
for i in range(N):
    if ((rf_probs[i,0] > 0.4 and rf_probs[i,0] < 0.6) and (logistic_probs[i,0] > 0.9 or logistic_probs[i,0] < 0.1)):
        if preds_logistic[i] == 1:
            preds_new[i] = -1
        elif preds_logistic[i] == -1:
            preds_new[i] = 1
    else:
        preds_new[i] = preds[i]
    

In [ ]:
preds_new_quiz = sp.zeros(N_quiz)
for i in range(N_quiz):
    if ((rf_probs_quiz[i,0] > 0.4 and rf_probs_quiz[i,0] < 0.6) and (logistic_probs_quiz[i,0] > 0.9 or logistic_probs_quiz[i,0] < 0.1)):
        if preds_logistic_quiz[i] == 1:
            preds_new_quiz[i] = -1
        elif preds_logistic_quiz[i] == -1:
            preds_new_quiz[i] = 1
    else:
        preds_new_quiz[i] = preds_quiz[i]

In [ ]:
error1 = 0
error2 = 0
for i in range(N):
    if preds_new[i] != labels[i]:
        error1 += 1
    if preds[i] != labels[i]:
        error2 += 1
rate1 = float(error1)/N
rate2 = float(error2)/N
print rate1, rate2

In [ ]:

submission = pd.DataFrame({"Prediction": preds_new_quiz})
submission.index += 1
submission.to_csv("submission9.csv", index_label="Id")
print 4

In [ ]:
#data_mat = data_in.as_matrix()
#misclass_list = []
#count0 = 0
#count1 = 0
#count2 = 0
#count3 = 0
#count4 = 0

#for i in range(N):
#    if ((rf_probs[i,0] > 0.4 and rf_probs[i,0] < 0.6) and (logistic_probs[i,0] > 0.9 or logistic_probs[i,0] < 0.1)):
#            count2 +=1
#            print preds_logistic[i],labels[i]
#            if preds_logistic[i] != labels[i]:
#                count3 += 1
#            if preds[i] != labels[i]:
#                count4 += 1
        
#print count0, count1, count2, count3, count4

In [ ]:
#data_2 = quiz_sparse[lowprob_list,:]
#labels_2 = labels[lowprob_list]
#print len(labels_2)

In [ ]:
#RandomForest_2 = RandomForestClassifier(n_estimators=40, criterion='gini', max_features='sqrt', n_jobs=-1,
#                                       min_samples_split = 4)
#scores_forest_2 = cross_validation.cross_val_score(RandomForest_2, data_2, labels_2, cv=5)
#RandomForest_2.fit(data_2, labels_2)
#preds_2 = RandomForest_2.predict(data_2)
#print scores_forest_2

In [ ]:
#for i in range(len(lowprob_list)):
#    preds_new[lowprob_list[i]] = preds_2[i]


In [ ]:
#submission = pd.DataFrame({"Prediction": preds_new_quiz})
#submission.index += 1
#submission.to_csv("submission8.csv", index_label="Id")
#print 4

In [ ]:
# use SelectKBest to reduce dimension of data
#KBest = SelectKBest(f_classif, k=100)
#data_best = KBest.fit_transform(data_sparse, labels)
#ind_best = KBest.get_support(indices=True)
#quiz_best = quiz_sparse[:,ind_best]
#print 1

In [ ]:
#sklearn.decomposition.PCA(n_components=None, copy=True, whiten=False)[source]
#pca = PCA(n_components=50)
#pca.fit(data)
#data_new = pca.transform(data)
#print data_new.shape
#print(pca.explained_variance_ratio_) 

In [ ]:

# best so far
#RandomForest_3 = RandomForestClassifier(n_estimators=40, criterion='gini', max_features='sqrt', n_jobs=-1)
#scores_forest = cross_validation.cross_val_score(RandomForest_3, data_sparse, labels, cv=5)
#RandomForest_3.fit(data_sparse, labels)
#print scores_forest

#preds = RandomForest_3.predict(quiz_sparse)
#submission = pd.DataFrame({"Prediction": preds})
#submission.index += 1
#submission.to_csv("submission4.csv", index_label="Id")
#print 4


In [ ]:
#Logistic = LogisticRegression(C=1.0)
#scores_logistic = cross_validation.cross_val_score(Logistic, data_best, labels, cv=5)
#print scores_logistic


In [ ]:
#bdt_real = AdaBoostClassifier(
#    base_estimator = RandomForest_3,
#    n_estimators=200,
#    learning_rate=1)
#scores_bdt = cross_validation.cross_val_score(bdt_real, data_best, labels, cv=5)
#bdt_real.fit(data_best, labels)
#print scores_bdt
#print 6

In [ ]:
#bdt_real.fit(data_best, labels)
#preds = bdt_real.predict(quiz_best)
#submission = pd.DataFrame({"Prediction": preds})
#submission.index += 1
#submission.to_csv("submission4.csv", index_label="Id")
#print 5